In [ ]:
!pip install transformers gradio sentencepiece


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import gradio as gr
import re

# 1. 더 나은 한국어 모델 로드 (KLUE-BERT 사용)
MODEL_NAME = "klue/bert-base"  # 또는 "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# 2. 텍스트 전처리 함수
def preprocess_text(text):
    """한국어 텍스트 전처리"""
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 정리 (필요에 따라 조정)
    text = re.sub(r'[^\w\s가-힣.,!?]', '', text)
    return text.strip()

# 3. 개선된 청킹 함수
def chunk_text_korean(text, max_tokens=50, overlap=10):
    """한국어 텍스트를 토큰 기반으로 청킹"""
    # 텍스트 전처리
    text = preprocess_text(text)

    # 토큰화 (special tokens 포함)
    tokens = tokenizer.encode(text, add_special_tokens=True, truncation=False)
    chunks = []

    # 특수 토큰 제거 ([CLS], [SEP] 등)
    if tokens[0] == tokenizer.cls_token_id:
        tokens = tokens[1:]
    if tokens[-1] == tokenizer.sep_token_id:
        tokens = tokens[:-1]

    # 청킹 수행
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i:i+max_tokens]
        chunks.append(chunk)

    # 디코딩 시 특수 토큰 제거 및 정리
    decoded_chunks = []
    for chunk in chunks:
        decoded = tokenizer.decode(
            chunk,
            skip_special_tokens=True,  # 특수 토큰 제거
            clean_up_tokenization_spaces=True  # 토큰화 공백 정리
        )
        # 추가 정리
        decoded = re.sub(r'\s+', ' ', decoded).strip()
        decoded_chunks.append(decoded)

    return decoded_chunks

# 4. 임베딩 생성 함수 (개선)
def get_embedding(chunk_tokens):
    """청크에 대한 임베딩 생성"""
    # 특수 토큰 추가
    input_ids = torch.tensor([[tokenizer.cls_token_id] + chunk_tokens + [tokenizer.sep_token_id]])
    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # [CLS] 토큰의 임베딩 사용 (또는 평균 풀링)
    embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] token
    # embeddings = outputs.last_hidden_state.mean(dim=1)  # 평균 풀링 대안

    return embeddings.numpy()

# 5. 토큰 분석 함수 추가
def analyze_tokens(text):
    """토큰 분석 및 UNK 토큰 확인"""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    token_analysis = []

    for token in tokens:
        decoded = tokenizer.decode([token])
        if token == tokenizer.unk_token_id:
            token_analysis.append(f" [UNK] (ID: {token})")
        else:
            token_analysis.append(f" '{decoded}' (ID: {token})")

    return token_analysis

# 6. Gradio 인터페이스 함수 (개선)
def process_text(text, max_tokens, overlap):
    if not text.strip():
        return "텍스트를 입력해주세요."

    # 토큰 분석
    token_analysis = analyze_tokens(text)
    unk_count = sum(1 for analysis in token_analysis if "[UNK]" in analysis)

    # 청킹 수행
    chunks = chunk_text_korean(text, max_tokens, overlap)
    results = []

    # 토큰 분석 결과 추가
    results.append(f" **토큰 분석**")
    results.append(f"- 총 토큰 수: {len(token_analysis)}")
    results.append(f"- UNK 토큰 수: {unk_count}")
    if unk_count > 0:
        results.append(f" UNK 토큰이 {unk_count}개 발견되었습니다.")
    results.append("")

    # 청킹 결과
    results.append(f" **청킹 결과** (총 {len(chunks)}개)")
    results.append("")

    for idx, chunk in enumerate(chunks):
        chunk_tokens = tokenizer.encode(chunk, add_special_tokens=False)
        try:
            emb = get_embedding(chunk_tokens)
            results.append(f"**청킹 {idx+1}:**")
            results.append(f"```\n{chunk}\n```")
            results.append(f" 토큰 수: {len(chunk_tokens)} | 임베딩 shape: {emb.shape}")
            results.append("")
        except Exception as e:
            results.append(f"**청킹 {idx+1}:**  오류 발생 - {str(e)}")
            results.append("")

    return "\n".join(results)

# 7. 예제 텍스트 생성
def get_example_text():
    return """
안녕하세요! 이것은 한국어 텍스트 청킹을 테스트하기 위한 예제 문장입니다.
자연어 처리에서 긴 텍스트를 작은 단위로 나누는 것을 청킹이라고 합니다.
청킹은 BERT와 같은 트랜스포머 모델의 최대 토큰 길이 제한을 해결하는 중요한 기법입니다.
한국어는 교착어의 특성상 형태소 분석이 복잡하며, 다양한 어미 변화가 있어 토큰화 과정에서 주의가 필요합니다.
"""

# 8. Gradio UI (개선)
demo = gr.Interface(
    fn=process_text,
    inputs=[
        gr.Textbox(
            label="한국어 입력 텍스트",
            lines=6,
            placeholder="긴 한국어 문장을 입력하세요...",
            value=get_example_text()
        ),
        gr.Slider(20, 200, value=50, step=10, label="최대 토큰 수 (max_tokens)"),
        gr.Slider(0, 50, value=10, step=5, label="오버랩 토큰 수 (overlap)")
    ],
    outputs=gr.Textbox(label="청킹 결과 및 분석", lines=20),
    title="🔍 한국어 텍스트 청킹 & 임베딩 분석기",
    description="""
    **기능:**
    - 한국어 텍스트의 토큰 분석 (UNK 토큰 감지)
    - 지능형 텍스트 청킹
    - KLUE-BERT 기반 임베딩 생성

    **개선사항:**
    - UNK 토큰 최소화를 위한 전처리
    - 더 나은 한국어 지원 모델 사용
    - 상세한 토큰 분석 제공
    """,
    examples=[
        [get_example_text(), 50, 10],
        ["인공지능과 머신러닝 기술이 급속도로 발전하고 있습니다.", 30, 5],
        ["한국어 자연어 처리는 형태소 분석, 구문 분석, 의미 분석 등 여러 단계를 거칩니다.", 40, 8]
    ]
)

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d5e292686cb97cf65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
